In [ ]:
!pip install transformers datasets evaluate rouge_score
!pip install peft

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=dd7c5eab1a8d64cd8886411fd1d87e9008e421a9310e0cecc64b8ce5f6fa492f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 4.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## 將數據集分割為訓練集和測試集

In [ ]:
from datasets import load_dataset
full_dataset = load_dataset("JamesLin0219/cyberSummarization", split = "train")
dataset = full_dataset.train_test_split(test_size = 0.2)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/403 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40824 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', '__index_level_0__'],
        num_rows: 32659
    })
    test: Dataset({
        features: ['text', 'summary', '__index_level_0__'],
        num_rows: 8165
    })
})

## load a T5 tokenizer to process text and summary

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, DataCollatorForSeq2Seq
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## 預處理數據

In [ ]:
prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/32659 [00:00<?, ? examples/s]

Map:   0%|          | 0/8165 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/32659 [00:00<?, ? examples/s]

##  Setup the PEFT/LoRA model for Fine-Tuning

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)
peft_model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
print(peft_model.print_trainable_parameters())

trainable params: 3,538,944 || all params: 251,116,800 || trainable%: 1.4093
None


## Train PEFT Adapter

In [ ]:
training_args = TrainingArguments(
    output_dir="t5_lora",
    auto_find_batch_size=True,
    learning_rate=2e-5,
    num_train_epochs=3,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    logging_strategy = "epoch",
)

# Add padding to the data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=peft_model,
    padding='longest',  # Add padding to the longest sequence in the batch
    return_tensors='pt'
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer = tokenizer,
    data_collator=data_collator
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

In [ ]:
peft_model.save_pretrained("t5_lora")
tokenizer.save_pretrained("t5_lora")

In [ ]:
peft_model.push_to_hub("JamesLin0219/t5LoRASummarization")

## Inference

In [ ]:
text = "summarize: In today's fast-evolving digital landscape, cybersecurity has become a cornerstone of organizational resilience. As cyber threats grow increasingly sophisticated, the demand for skilled cybersecurity professionals has never been higher. Whether you're a seasoned cyber professional or just starting your journey, signing up for the GIAC Newsletter ensures you're always informed and equipped for the evolving landscape of cybersecurity."

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5_lora")
inputs = tokenizer(text, return_tensors="pt").input_ids

In [ ]:
from peft import PeftConfig, PeftModel
config = PeftConfig.from_pretrained("t5_lora")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
finetuned_model = PeftModel.from_pretrained(model, "t5_lora")
outputs = model.generate(inputs, max_new_tokens=500, do_sample=True)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

summarization_config = PeftConfig.from_pretrained("JamesLin0219/t5LoRASummarization")
summarization_model = PeftModel.from_pretrained(model, "JamesLin0219/t5LoRASummarization")
summarization_tokenizer = AutoTokenizer.from_pretrained(model_name)

text = """summarize: In recent years, cybercriminals have increasingly exploited vulnerabilities in widely-used IT and security tools, leading to major security incidents. For instance, a zero-day vulnerability in Ivanti enterprise VPNs was recently exploited, allowing attackers to deploy a backdoor named ‘DSLog’. Similarly, a remote code execution vulnerability in TeamCity enabled attackers, suspected to be from the APT29 group, to infiltrate systems by installing malicious SSH certificates, using PowerShell to download and execute malicious DLLs, and maintaining persistence through scheduled tasks. Another example is the Fortra GoAnywhere MFT vulnerability, which ransomware groups like LockBit and Cl0p exploited to execute remote code, resulting in significant attacks, particularly in the healthcare sector."""
inputs = summarization_tokenizer(text, return_tensors="pt").input_ids
outputs = summarization_model.generate(input_ids = inputs, max_new_tokens = 1024, do_sample = True)


In [ ]:
summarization_tokenizer.decode(outputs[0], skip_special_tokens=True)